Un notebook propre et commenté (introduction,
analyses, conclusion, etc) contenant : le procédé de
développement de votre outil, du prétraitement à la
modélisation des données, en passant par l’analyse
exploratoire.

Pensez à répondre à la problématique.